<a href="https://colab.research.google.com/github/lenonborges/Optimizing-Restaurant-s-Search/blob/main/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing the search for restaurants 

## Summary
-Introduction

-Business Problem

-Data to solve the problem

-Results

-Discussion

-Conclusion

## Introduction
This project's goal is to deliver the n best food venues (considering the best quality x price relation) near you. We will get the data for some food venues places in Rio de Janeiro since the Foursquare API limits the response objects.

As human beings we tend to get annoyed of going to the same places over and over, so it would be nice if we got a list of the n best options in a price vs quality view.
Then we can choose amongst a list which best suit for us on that particular moment. 

## Business Problem 
Nowadays, on May 2021, Google maps allows filter for price and quality given a particular area on the map. It also gives you a list of places with filter for quality levels in steps of 0.5 points of difference. Although, Google maps already has great filters such as: Romantic, Cozy, Fun and others. However there is no filter for a good and not expensive place to go.  

Therefore, the audience of this project is every person who wants to get a new, good and affordable place to eat.

## Data to solve the problem
We'll use the Foursquare API to gather data for venues in a given City. The data is such as location, latitude, longitude, name, price, rating and others. 

With these data we will construct a factor given weights for price and rating, actually 30% for price and 70% for rating. Where price has a negative effect and rating has a positive effect on our factor.

We will also use the Follium library to plot that found venues on map. 







In [ ]:
pip install requests

In [ ]:
import requests

In [ ]:
ID = ''
SECRET = ''
cidade = 'Rio de Janeiro'
vers = '20210324'
oauth= ''
categoria = '4bf58dd8d48988d110941735' #italian
#diner '4bf58dd8d48988d147941735' #food '4d4b7105d754a06374d81259' #japones '4bf58dd8d48988d111941735' australian '4bf58dd8d48988d169941735'
#french '4bf58dd8d48988d10c941735'
#Tech Startup 4bf58dd8d48988d125941735
#italian 4bf58dd8d48988d110941735

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search/?near={}&categoryId={}&cliend_id={}&client_secret={}&oauth_token={}&v={}'.format(cidade, categoria, ID, SECRET,oauth, vers)
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '60ba2c9bdb321b25f77bbf31'},
 'notifications': [{'item': {'unreadCount': 0}, 'type': 'notificationTray'}],
 'response': {'confident': False,
  'geocode': {'feature': {'cc': 'BR',
    'displayName': 'Rio de Janeiro, RJ, Brazil',
    'geometry': {'bounds': {'ne': {'lat': -22.74873707502691,
       'lng': -43.10168807024157},
      'sw': {'lat': -23.07659699961192, 'lng': -43.796475958003505}},
     'center': {'lat': -22.90278, 'lng': -43.2075}},
    'highlightedName': '<b>Rio de Janeiro</b>, RJ, Brazil',
    'id': 'geonameid:3451190',
    'longId': '72057594041379126',
    'matchedName': 'Rio de Janeiro, RJ, Brazil',
    'name': 'Rio de Janeiro',
    'slug': 'rio-de-janeiro',
    'woeType': 7},
   'parents': [],
   'what': '',
   'where': 'rio de janeiro'},
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Rest

In [ ]:
from pandas.io.json import json_normalize

# assign relevant part of JSON to venues
venues = result['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4f3d15cae4b0fc2b1f7cef6c,Ristorante Casa do Sardo,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1622813851,False,"R. S. Cristóvão, 405",-22.903560,-43.218618,"[{'label': 'display', 'lat': -22.9035596606659...",20940-001,BR,Rio Janeiro,RJ,Brasil,"[R. S. Cristóvão, 405, Rio Janeiro, RJ, 20940-...",NaN,NaN,NaN
1,52eeb3d1498ebb78f4850b86,Fabene Massas,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1622813851,False,"Rua Campo de São Cristovão, 404",-22.899197,-43.223186,"[{'label': 'display', 'lat': -22.8991970679112...",NaN,BR,Rio de Janeiro,RJ,Brasil,"[Rua Campo de São Cristovão, 404, Rio de Janei...",NaN,NaN,NaN
2,5cc3a98f9e0d540037143040,Posì Mozza & Mare,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1622813851,False,"R. Aníbal de Mendonça, 158",-22.982335,-43.211056,"[{'label': 'display', 'lat': -22.982335, 'lng'...",22410-050,BR,Rio de Janeiro,RJ,Brasil,"[R. Aníbal de Mendonça, 158 (R. Redentor), Rio...",R. Redentor,NaN,NaN
3,562925ab498ee29483d32d14,Manjerona Cantina Italiana e Pizzaria,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1622813851,False,"Estr. do Pau-Ferro, 431",-22.927391,-43.343301,"[{'label': 'display', 'lat': -22.9273905148276...",22743-050,BR,Rio de Janeiro,RJ,Brasil,"[Estr. do Pau-Ferro, 431, Rio de Janeiro, RJ, ...",NaN,Jacarpaguá,NaN
4,5324824c11d2d2f5f87b9d5a,Don Carlo's Pizza Gourmet,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1622813851,False,"R. Francisco Pimentel, 186",-22.904127,-43.122717,"[{'label': 'display', 'lat': -22.9041270448142...",24210-465,BR,Niterói,RJ,Brasil,"[R. Francisco Pimentel, 186, Niterói, RJ, 2421...",NaN,NaN,85372774


In [ ]:
lojas_rating = []
lojas_price = []
for j in df['id']:
  loja = j
  url = 'https://api.foursquare.com/v2/venues/{}?&cliend_id={}&client_secret={}&oauth_token={}&v={}'.format(loja, ID, SECRET,oauth, vers)
  result2 = requests.get(url).json()
  try:
    rat = result2['response']['venue']['rating']
    lojas_rating.append(rat)
    pri = result2['response']['venue']['price']['tier']
    lojas_price.append(pri)
  except:
    rat = 'N/A'
    lojas_rating.append(rat)
    pri = 'N/A'
    lojas_price.append(pri)


In [ ]:
len(lojas_rating)

30

In [ ]:
len(df['id'])

30

In [ ]:
df['price'] = lojas_price
df['rating'] = lojas_rating

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df.price

0     2
1     2
2     2
3     2
4     2
5     2
6     2
7     3
8     2
9     2
10    2
11    2
12    2
13    3
14    2
15    2
16    2
17    2
18    2
19    2
20    2
21    3
22    2
23    3
24    3
25    3
26    2
27    4
28    3
29    2
Name: price, dtype: int64

In [ ]:
df3 = df[(df.rating != 'N/A') & (df.price != 'N/A')]

In [ ]:
df3['fator'] = (df3['rating']*0.7) - (df3['price']*0.3)

## Results
Results are shown on a DataFrame and on a map where the best five places in a quality vs price relation are plotted with a blue circle and everyone else with a red circle.

## Discussion
Since we are using the free version of the API we will get less results than it would be in a premium version. It affects our results, but is easily solved if desired. 



In [ ]:
df4 = df3[['name','price','rating','fator','location.address','location.city','location.formattedAddress','location.lat',	'location.lng']]

In [ ]:
df4 = df4.sort_values(['fator'], ascending=False).reset_index(drop=True)
df4

NameError: ignored

In [ ]:
df4.index[9]

15

In [ ]:
import folium
from folium import vector_layers

In [ ]:
REST = folium.Map(location=[df4['location.lat'][0],	df4['location.lng'][0]], zoom_start=13)

for i in range(0, len(df4['name'])):
  # create a feature group
  re = folium.map.FeatureGroup()

  if i>=4:
    #style the feature group
    re.add_child(folium.CircleMarker([df4['location.lat'][i],df4['location.lng'][i]], color= "red", fill_color="red",radius=6))
  else:
    re.add_child(folium.CircleMarker([df4['location.lat'][i],df4['location.lng'][i]], color = "blue", fill_color="blue",radius=6))
  
  #add the feature to the map
  REST.add_child(re)
  
  #label the marker
  label = '{}º, {}, {}'.format(df4.index[i]+1,df4['name'][i],df4['fator'][i])
  folium.Marker([df4['location.lat'][i],df4['location.lng'][i]], popup=label).add_to(REST)


REST

## Conclusion
That project aims to deliver a solution desired for many people which is find good and affordable places to go to. Which we showed that is possible to be done.